# Evaluating Performance of Llama 3.1 8B with Nemo Evaluator OSS

[NeMo Evaluator](https://github.com/NVIDIA-NeMo/Evaluator/tree/main) is an open-source platform for robust, reproducible, and scalable evaluation of Large Language Models. It can be used to run the most popular academic benchmarks by executing open-source Docker containers. 

We will use this library to evaluate [Llama 3.1 8B](https://build.nvidia.com/meta/llama-3_1-8b-instruct), which will be deployed as a NIM in the Lepton service.

The benchmark used in this notebook will be [MMLU](https://huggingface.co/datasets/cais/mmlu), a QA dataset for multitask understanding.

[This blog post](https://developer.nvidia.com/blog/mastering-llm-techniques-evaluation/) provides more details into LLM evaluation and how to choose the right benchmarks and techniques.

### Requirements
* GPUs: minimum 1 NVIDIA GPU (default is H200 - you will need to change the variable resource_shape in *Step 3.3: Deployment* otherwise)
* <u>NGC API key</u>: obtained from [Nvidia](https://build.nvidia.com/)
* <u>Hugging Face [access token](https://huggingface.co/docs/hub/en/security-tokens)</u>: this will be used to download gated models or datasets
* <u>Lepton token</u>: this notebook is made to run **locally**, which means you need a token to login to your Lepton workspace so that the endpoint and the batch jobs are deployed. In your lepton workspace, go to *Settings* and then *Tokens*
* <u>Image Registry Secret</u>: for image pulling, to get the evaluation tasks. If your workspace does not have this, go to *Settings -> Registries -> New Registry Auth -> NVIDIA* and set it up per the [documentation](https://docs.nvidia.com/dgx-cloud/lepton/features/workspace/registry/#nvidia)

### Objectives

The goal of this notebook is to demonstrate the usage of NeMo Evaluator OSS in Lepton. 

NeMo Evaluator can be used locally, with Slurm or with Lepton. Evaluation requires a model to be deployed first, and then a series of requests are made to its endpoint. While this notebook shows how to deploy the model with NIM, the scripts in [NeMo Evaluator](https://github.com/NVIDIA-NeMo/Evaluator/tree/main) allow to deploy the model via VLLM or use an already deployed endpoint.

Model evaluation can be performed in many different ways, where the most common one is to get the model's answers to questions in a benchmark, and compare them to a ground truth or have a separate, more powerful LLM judge them (LLM-as-a-judge). There are many different benchmarks available to use in NeMo Evaluator: this [GitHub link](https://github.com/NVIDIA-NeMo/Evaluator/tree/main/tutorials) provides tutorials and this [documentation page](https://nv-eval-platform-dl-joc-competitive-evaluation-c0c268c1aead4fd0.gitlab-master-pages.nvidia.com/benchmarks_doc) contains more details into the different benchmarks included and how to run them.

### Step 1: Create your environment

For this notebook, you will need to install leptonai and NeMo Evaluator OSS. These can be done with the following commands:

In [ ]:
!pip install leptonai nemo-evaluator-launcher

### Step 2: Connect to Lepton

Now, you need to connect to your lepton workspace using the token you obtained. <span style="color:blue">Paste your token below</span>, and run the cell

In [ ]:
!LEP_TOKEN="YOUR_TOKEN_HERE" && lep login -c $LEP_TOKEN

You should see **LEPTON** in big green font, and text indicating you're logged in.

### Step 3: Build your NeMo Evaluator script with your keys

In this section, we will be creating an Evaluation runner configuration file, similar to the Lepton example found in [GitHub](https://github.com/NVIDIA-NeMo/Evaluator/blob/main/packages/nemo-evaluator-launcher/examples/lepton_nim_llama_3_1_8b_instruct.yaml).

By running the configuration, you will:
1. Deploy the specified NIM container to a Lepton endpoint
2. Wait for the endpoint to be ready
3. Run evaluation tasks as parallel Lepton jobs that connect to the deployed NIM
4. Clean up the endpoint when done (on failure) or remind you to clean up (on success)

Let's create the configuration yaml part by part.

##### Step 3.1: Defaults

In [ ]:
defaults = f"""

defaults:
  - execution: lepton/default
  - deployment: nim
  - _self_
  
"""

The only thing of note here is the *deployment* section, which is defined as **nim** for this notebook. This would change depending on your model deployment.

##### Step 3.2: Execution

Let's now write the **execution** configurations. For this, you will need the id of your Lepton node. 

You can obtain the node id by going to your Lepton Dashboard, click on **Nodes** and then click on the node box you see. You will find the node id on the url link, right after */node-groups/detail/dedicated* and in between *'/'* (example: *nv-int-multiteam-nebius-h200-01-mjgbgffo*). 
<span style="color:blue">Write your node id in the cell below.</span>

In [ ]:
lepton_node_id = "YOUR_NODE_ID_HERE"

<span style="color:blue">Set your Lepton storage path</span>, where the results will be saved:

In [ ]:
lepton_storage_path = "/EU-Model-Builder-SAs/user_homes/${oc.env:USER}/nemo-evaluator-launcher-workspace"

Additionally, you will need your NGC and your Hugging Face tokens, as well as your registry secret. <span style="color:blue">Set them here:</span>

In [ ]:
ngc_key = "YOUR_NGC_KEY_HERE"
hf_token = "YOUR_HF_TOKEN_HERE"
registry_secret = "YOUR_REGISTRY_SECRET_HERE"

In [ ]:
execution = f"""

execution:
  output_dir: lepton_nim_llama_3_1_8b_results

  evaluation_tasks:
    resource_shape: "cpu.large"  # Evaluation tasks require only CPU resources
    timeout: 3600  # Override default 3600 timeout (this is how long we wait for the endpoint to be ready)

  lepton_platform:
    deployment:
      node_group: {lepton_node_id}

      platform_defaults:
        image_pull_secrets:
          - "{registry_secret}"  # Secret to pull private images from NGC registry

    tasks:
      api_tokens:
      - value_from:
          token_name_ref: "ENDPOINT_API_KEY"  # Token to access the model endpoint
          
      env_vars:
        HF_TOKEN: "{hf_token}"

      # Node group for evaluation tasks
      node_group: {lepton_node_id.split(lepton_node_id.split('-')[-1])[0][:-1]}
      
      # Storage mounts for task execution
      mounts:
        # Main workspace mount
        - from: "node-nfs:lepton-shared-fs"
          path: {lepton_storage_path}
          mount_path: "/workspace"
          
"""

##### Step 3.3: Deployment

Now let's look at the NIM-specific deployment configurations.

In [ ]:
deployment = f"""

deployment:
  # NIM container configuration
  image: nvcr.io/nim/meta/llama-3.1-8b-instruct:1.8.6
  served_model_name: meta/llama-3.1-8b-instruct

  # Lepton-specific deployment settings
  lepton_config:
    endpoint_name: llama-3-1-8b  # Base name for Lepton endpoint
    resource_shape: gpu.1xh200 # GPU shape for the endpoint
    min_replicas: 1
    max_replicas: 1

    api_tokens:
      - value_from:
          token_name_ref: "ENDPOINT_API_KEY"  # Token for the model endpoint (must be the same as the api_tokens in the Execution section)

    # Auto-scaling settings
    auto_scaler:
      scale_down:
        no_traffic_timeout: 3600
        scale_from_zero: false

    # Environment variables for NIM container
    envs:
      # Direct values
      OMPI_ALLOW_RUN_AS_ROOT: "1"
      OMPI_ALLOW_RUN_AS_ROOT_CONFIRM: "1"

      HF_TOKEN: "{hf_token}"
      
      NGC_API_KEY: "{ngc_key}"

    # Storage mounts for model caching
    mounts:
      enabled: true
      cache_path: {lepton_storage_path}/.cache
      mount_path: "/opt/nim/.cache"

"""

##### Step 3.4: Evaluation

Now we can move to the Evaluation section. Here, we can choose one or more tasks to run the model in. 

In this case, we are choosing to run the MMLU task from *simple_evals*. A task is run simply by defining the task name, and you can choose to run multiple tasks by adding more "-name" keys. The number of batch jobs deployed in Lepton will be the same as the number of tasks defined here, since each task corresponds to a separate image.

Defining the task name is enough to run it with defauls, but you can choose as many overrides as you wish by following the [documentation](https://nv-eval-platform-dl-joc-competitive-evaluation-c0c268c1aead4fd0.gitlab-master-pages.nvidia.com/benchmarks_doc). In this case, we have increased the model request timeout and the number of saved requests and responses for cache, but you can also tinker with model-specific parameters like temperature or top p.

In [ ]:
evaluation = f"""

evaluation:
  # Evaluation tasks to run
  tasks:
    - name: simple_evals.mmlu
      overrides: # task-specific overrides
        config.params.request_timeout: 3200
        target.api_endpoint.adapter_config.max_saved_requests: 100
        target.api_endpoint.adapter_config.max_saved_responses: 100 
    
"""

##### Step 3.5: Saving the yaml file

All the components are ready for the file to be saved. <span style="color:blue">Define the directory where the file is saved:</span>

In [ ]:
%env CURR_DIR=eval

In [ ]:
!mkdir -p $CURR_DIR

In [ ]:
import os

with open(f"{os.environ['CURR_DIR']}/lepton_nim_evaluation.yaml", "w") as f:
    f.write(defaults + execution + deployment + evaluation)

## Step 4: Deploy the Evaluation Job

Now that we have created the yaml file, it's time to deploy it in Lepton.
Once the deployment is ready, you can monitor it in the Lepton UI:
- Deployment status: UI/Endpoints
- Evaluation jobs: UI/Batch Jobs

The command in the following cell deploys the job; **make sure you input the paths for your current directory (where *lepton_nim_evaluation.yaml* is saved) and the path where you want results stored in locally**.

In [ ]:
%env RESULTS_DIR=results

In [ ]:
!nemo-evaluator-launcher run --config-dir $CURR_DIR --config-name lepton_nim_evaluation --override execution.output_dir=$RESULTS_DIR

______________________

That's it! You have managed to deploy a NIM model and run an evaluation task on Lepton. Check the documentation to learn how to run other tasks ✓ 